In [1]:
from azureml.core import Workspace, Experiment, Model
import joblib

ws = Workspace.from_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: capstone
Azure region: westeurope
Subscription id: 72f46e0e-1451-4b79-92cd-fc8f7797bda7
Resource group: test-Jesse


In [2]:
# I've ran five experiments. below code retrieves the main experiment run and the best run. It also prints the score of the best run

# Dictionary to map the experiments to better descriptive names
explanation = { 'hyperdrive-xgboost' : 'hyperdrive xgboost with bandit policy',
                'udacity-project-hyperparam-bayes' : 'hyperdrive xgboost with bayesian sampling',
                'automl': 'AutoML on same dataset as hyperdrive',
                'AutoML-onlylogtransform': 'AutoML on original dataset (except log transform target)',
                'automl-kaggle-orgininal': 'AutoML on original dataset (no log transform target)'
}

def get_score(run):
    print(float(run.get_metrics()['rmse']) if run.status == 'Completed' else None)
    return float(run.get_metrics()['rmse']) if run.status == 'Completed' else None

def get_main_run(exp_name):
    exp = Experiment(workspace=ws, name=exp_name)
    return list(exp.get_runs())[0]

def get_best_run(name, run):
    print(explanation[name], end = '\t')
    if run.type == 'hyperdrive':
        best_run = run.get_best_run_by_primary_metric()
        print(round(best_run.get_metrics()['rmse'],4))
    elif run.type == 'automl':
        best_run = run.get_best_child()
        print(round(best_run.get_metrics()['root_mean_squared_error'],4))
    else:
        print('wrong type')
    return best_run

main_runs = {exp_name : get_main_run(exp_name) for exp_name in list(explanation.keys())}
best_runs = {name: get_best_run(name, run) for name, run in main_runs.items()}
    

hyperdrive xgboost with bandit policy	0.1255
hyperdrive xgboost with bayesian sampling	0.127
AutoML on same dataset as hyperdrive	0.1167
AutoML on original dataset (except log transform target)	0.1248
AutoML on original dataset (no log transform target)	26738.5013


## Register best AutoML model

In [9]:
model = main_runs['automl'].register_model(model_name = 'automl')
# below code to download model
# model.download(target_dir='models/automl', exist_ok = True)
# model = joblib.load('models/automl/model.pkl')


## Register best Hyperdrive model

In [13]:
best_runs['hyperdrive-xgboost']

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-xgboost,HD_091ce256-d06c-47bb-b5fb-99f68e45a6a3_125,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
best_runs['hyperdrive-xgboost'].register_model(model_name = 'hyperdrive', model_path='./outputs')

Model(workspace=Workspace.create(name='capstone', subscription_id='72f46e0e-1451-4b79-92cd-fc8f7797bda7', resource_group='test-Jesse'), name=hyperdrive, id=hyperdrive:1, version=1, tags={}, properties={})